In [1]:
import sys  
sys.path.insert(0, '/Users/arturo/Downloads/')

from Funciones import *

In [2]:
import pyspark
from pyspark.sql import SparkSession
from Funciones import *
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
# from pyspark.sql.types import ArrayType, DoubleType, BooleanType
# from pyspark.sql.functions import col,array_contains



In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/20 11:57:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Datos

In [4]:
fi4 = spark.read.csv("DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", sep=";", header=True)
a12 = spark.read.csv("DATA_RELACION_ENTIDADES_MAESTRO.csv", sep=";", header=True)
fi5 = spark.read.csv("DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv", sep=";", header=True)

# Cruce 1

In [5]:
cruce1 = fi5.join(a12,fi5.Id ==  a12.Id_NIVEL_0,"outer")

In [6]:
cruce1 = cruce1.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(cruce1.NOMBRE_ENTIDAD_NIVEL_1),
                               cruce1.Centro_Norm
                           )) 

# DATA_SIC_ENTIDADES_CENTROS_PB

In [59]:
data = cruce1.filter("Id_NIVEL_0 is NULL")\
             .withColumn("Entidad_Norm",col("Centro_Norm"))\
             .withColumn("Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_Entidad_Norm",col("Centro_Norm"))\
             .withColumn("List_Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_CIF",lit(None).cast(StringType()))

In [60]:
data.select("Entidad_Norm", "Provincia_Entidad",
            "List_Entidad_Norm","List_Provincia_Entidad",
            "List_CIF").show(truncate = False)

+---------------------------+-----------------+---------------------------+----------------------+--------+
|Entidad_Norm               |Provincia_Entidad|List_Entidad_Norm          |List_Provincia_Entidad|List_CIF|
+---------------------------+-----------------+---------------------------+----------------------+--------+
|agr asociacion cic nanogune|null             |agr asociacion cic nanogune|null                  |null    |
|agr asociacion cic nanogune|null             |agr asociacion cic nanogune|null                  |null    |
|agr asociacion cic nanogune|null             |agr asociacion cic nanogune|null                  |null    |
|agr asociacion cic nanogune|null             |agr asociacion cic nanogune|null                  |null    |
|agr asociacion cic nanogune|null             |agr asociacion cic nanogune|null                  |null    |
|agr asociacion cic nanogune|null             |agr asociacion cic nanogune|null                  |null    |
|asoc cic nanogune          

In [51]:
fi4.printSchema()

root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [43]:
fi5.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Entidad: string (nullable = true)
 |-- Origen_Solicitud: string (nullable = true)
 |-- Identificadores_Origen: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- CIF_validacion: string (nullable = true)
 |-- Centro: string (nullable = true)
 |-- Centro_Norm: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- Pais_Entidad: string (nullable = true)
 |-- Provincia_Centro: string (nullable = true)
 |-- PIC: string (nullable = true)
 |-- Tipo_Persona: string (nullable = true)
 |-- Entidad_Match: string (nullable = true)
 |-- Provincia_Match: string (nullable = true)
 |-- Match: string (nullable = true)



In [6]:
cruce1.columns

['Id',
 'Entidad',
 'Origen_Solicitud',
 'Identificadores_Origen',
 'Entidad_Norm',
 'CIF',
 'CIF_validacion',
 'Centro',
 'Centro_Norm',
 'Tipo',
 'Provincia_Entidad',
 'Pais_Entidad',
 'Provincia_Centro',
 'PIC',
 'Tipo_Persona',
 'Entidad_Match',
 'Provincia_Match',
 'Match',
 'Id_NIVEL_1',
 'Id_NIVEL_0',
 'ID_ENTIDAD_NIVEL_0',
 'ID_ENTIDAD_NIVEL_1',
 'Jerarquia',
 'NIF_COD_NIVEL_0',
 'ACRONIMO_NIVEL_0',
 'NOMBRE_ENTIDAD_NIVEL_0',
 'Nombre_Entidad_Mostrar_NIVEL_0',
 'TIPO_ENTIDAD_N1_1_NIVEL_0',
 'TIPO_ENTIDAD_N2_1_NIVEL_0',
 'DIRECCION_POSTAL_NIVEL_0',
 'COD_POSTAL_NIVEL_0',
 'COD_PROVINCIA_NIVEL_0',
 'PROVINCIA_NIVEL_0',
 'COD_CCAA_NIVEL_0',
 'CCAA_NIVEL_0',
 'ENLACE_WEB_NIVEL_0',
 'SOMMA_NIVEL_0',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_0',
 'ESTADO_x_NIVEL_0',
 'NIF_COD_NIVEL_1',
 'ACRONIMO_NIVEL_1',
 'NOMBRE_ENTIDAD_NIVEL_1',
 'Nombre_Entidad_Mostrar_NIVEL_1',
 'TIPO_ENTIDAD_N1_1_NIVEL_1',
 'TIPO_ENTIDAD_N2_1_NIVEL_1',
 'DIRECCION_POSTAL_NIVEL_1',
 'COD_POSTAL_NIVEL_1',
 'COD_PROVINCIA_NI

['Id',
 'CIF',
 'Entidad_Norm',
 'Provincia_Entidad',
 'ID_ENTIDAD',
 'NIF_COD',
 'ACRONIMO',
 'NOMBRE_ENTIDAD',
 'Nombre_Entidad_Mostrar',
 'TIPO_ENTIDAD_N1_1',
 'TIPO_ENTIDAD_N2_1',
 'DIRECCION_POSTAL',
 'COD_POSTAL',
 'COD_PROVINCIA',
 'PROVINCIA',
 'COD_CCAA',
 'CCAA',
 'ENLACE_WEB',
 'SOMMA',
 'TIPO_ENTIDAD_REGIONAL',
 'ESTADO_x',
 'CodigoInvente',
 'DenominacionSocial',
 'FormaJuridica_Codigo',
 'FormaJuridica_Descripcion',
 'NIF',
 'codigoDir3',
 'codigoOrigen',
 'Provincia_Codigo',
 'C_ID_UD_ORGANICA',
 'C_DNM_UD_ORGANICA',
 'C_ID_NIVEL_ADMON',
 'C_ID_TIPO_ENT_PUBLICA',
 'N_NIVEL_JERARQUICO',
 'C_ID_DEP_UD_SUPERIOR',
 'C_DNM_UD_ORGANICA_SUPERIOR',
 'C_ID_DEP_UD_PRINCIPAL',
 'C_DNM_UD_ORGANICA_PRINCIPAL',
 'B_SW_DEP_EDP_PRINCIPAL',
 'C_ID_DEP_EDP_PRINCIPAL',
 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL',
 'C_ID_ESTADO',
 'D_VIG_ALTA_OFICIAL',
 'NIF_CIF',
 'C_ID_AMB_PROVINCIA',
 'C_DESC_PROV',
 'CONTACTOS',
 'List_Entidad_Norm',
 'List_Provincia_Entidad',
 'List_CIF']

In [14]:
cruce1.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Entidad: string (nullable = true)
 |-- Origen_Solicitud: string (nullable = true)
 |-- Identificadores_Origen: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- CIF_validacion: string (nullable = true)
 |-- Centro: string (nullable = true)
 |-- Centro_Norm: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- Pais_Entidad: string (nullable = true)
 |-- Provincia_Centro: string (nullable = true)
 |-- PIC: string (nullable = true)
 |-- Tipo_Persona: string (nullable = true)
 |-- Entidad_Match: string (nullable = true)
 |-- Provincia_Match: string (nullable = true)
 |-- Match: string (nullable = true)
 |-- Id_NIVEL_1: string (nullable = true)
 |-- Id_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_1: string (nullable = true)
 |-- Jerarquia: string (nullable = true)
 |--

In [12]:
fi4.printSchema()


root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [21]:
a12.printSchema()

root
 |-- Id_NIVEL_1: string (nullable = true)
 |-- Id_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_1: string (nullable = true)
 |-- Jerarquia: string (nullable = true)
 |-- NIF_COD_NIVEL_0: string (nullable = true)
 |-- ACRONIMO_NIVEL_0: string (nullable = true)
 |-- NOMBRE_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- Nombre_Entidad_Mostrar_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1_NIVEL_0: string (nullable = true)
 |-- DIRECCION_POSTAL_NIVEL_0: string (nullable = true)
 |-- COD_POSTAL_NIVEL_0: string (nullable = true)
 |-- COD_PROVINCIA_NIVEL_0: string (nullable = true)
 |-- PROVINCIA_NIVEL_0: string (nullable = true)
 |-- COD_CCAA_NIVEL_0: string (nullable = true)
 |-- CCAA_NIVEL_0: string (nullable = true)
 |-- ENLACE_WEB_NIVEL_0: string (nullable = true)
 |-- SOMMA_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL_NIVEL_0: string (nullab